In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import re
import pickle
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
import os

In [ ]:
data=pd.read_excel('/content/gdrive/My Drive/data.xlsx')

In [ ]:
data.columns

Index(['Headline', 'Short', 'Source ', 'Time ', 'Publish Date'], dtype='object')

In [ ]:
del data['Source ']

In [ ]:
del data['Time ']

In [ ]:
del data['Publish Date']

In [ ]:
data_=pd.DataFrame(data)

In [ ]:
data=data_

In [ ]:
data.head()

,Headline,Short
0,4 ex-bank officials booked for cheating bank o...,The CBI on Saturday booked four former officia...
1,Supreme Court to go paperless in 6 months: CJI,Chief Justice JS Khehar has said the Supreme C...
2,"At least 3 killed, 30 injured in blast in Sylh...","At least three people were killed, including a..."
3,Why has Reliance been barred from trading in f...,Mukesh Ambani-led Reliance Industries (RIL) wa...
4,Was stopped from entering my own studio at Tim...,TV news anchor Arnab Goswami has said he was t...


In [ ]:
data.shape

(55104, 2)

In [ ]:
summary=data['Headline']
document=data['Short']

In [ ]:
document[75]

'IPS officer Himanshu Kumar was suspended on Saturday after he tweeted that seniors in the police department under the Adityanath government are targeting subordinates with Yadav surname. However, he later claimed that his comments were misinterpreted. After the UP police suspended Himanshu for indiscipline, he tweeted, &#34;Truth alone triumphs.&#34;'

In [ ]:
summary[75]

'IPS officer suspended for tweeting against Yogi govt in UP'

In [ ]:
document_processed=[]
summary_processed=[]

In [ ]:
for i in document:
  result = re.sub('[\W]+', ' ', i) 
  result= '<start> ' + result +' <end>'
  document_processed.append(result)
for i in summary:
  result= re.sub('[\W]+', ' ',i)
  result= '<start> ' + result + ' <end>'
  summary_processed.append(result)

In [ ]:
document[1]

'Chief Justice JS Khehar has said the Supreme Court will go paperless in six to seven months in a bid to save funds and make the judiciary eco-friendly. He further said the apex court will collect all the records electronically from the lower courts and the high courts so that there is no need to file hard copies.'

In [ ]:
document_processed[1]

'<start> Chief Justice JS Khehar has said the Supreme Court will go paperless in six to seven months in a bid to save funds and make the judiciary eco friendly He further said the apex court will collect all the records electronically from the lower courts and the high courts so that there is no need to file hard copies  <end>'

In [ ]:
document_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
document_tokenizer.fit_on_texts(document_processed)
document_tensor = document_tokenizer.texts_to_sequences(document_processed)

document_tensor = tf.keras.preprocessing.sequence.pad_sequences(document_tensor,
                                                         padding='post')

In [ ]:
summary_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
summary_tokenizer.fit_on_texts(summary_processed)
summary_tensor = summary_tokenizer.texts_to_sequences(summary_processed)

summary_tensor = tf.keras.preprocessing.sequence.pad_sequences(summary_tensor,
                                                         padding='post')

In [ ]:
# Creating training and validation sets using an 80-20 split
document_tensor_train, document_tensor_val, summary_tensor_train, summary_tensor_val = train_test_split(document_tensor, summary_tensor, test_size=0.2)

# Show length
print(len(document_tensor_train), len(summary_tensor_train), len(document_tensor_val), len(summary_tensor_val))

44083 44083 11021 11021


In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [ ]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  
  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  
  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
  pos_encoding = angle_rads[np.newaxis, ...]
    
  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
pos_encoding = positional_encoding(50, 512)
print (pos_encoding.shape)

plt.pcolormesh(pos_encoding[0], cmap='RdBu')
plt.xlabel('Depth')
plt.xlim((0, 512))
plt.ylabel('Position')
plt.colorbar()
plt.show()

In [ ]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [ ]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

In [ ]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [ ]:
print ("Input Language; index to word mapping")
convert(document_tokenizer, document_tensor_train[0])
print ()
#print ("Target Language; index to word mapping")
convert(summary_tokenizer, summary_tensor_train[0])

Input Language; index to word mapping
8 ----> <start>
166 ----> following
134 ----> bjp
3 ----> 39
12 ----> s
947 ----> victory
4 ----> in
1 ----> the
386 ----> uttar
179 ----> pradesh
318 ----> assembly
345 ----> elections
107 ----> party
569 ----> member
2360 ----> subramanian
1768 ----> swamy
17 ----> said
15 ----> that
28 ----> he
21 ----> will
392 ----> move
1 ----> the
354 ----> supreme
83 ----> court
10 ----> on
279 ----> march
405 ----> 21
13 ----> for
26 ----> an
1024 ----> early
1468 ----> hearing
5 ----> of
1 ----> the
712 ----> ram
8157 ----> mandir
219 ----> case
11 ----> 34
65 ----> i
4384 ----> expect
190 ----> now
66 ----> up
8596 ----> govt
21 ----> will
44 ----> not
6608 ----> oppose
443 ----> me
11 ----> 34
28 ----> he
171 ----> tweeted
1768 ----> swamy
36 ----> had
95 ----> earlier
17 ----> said
11 ----> 34
154 ----> there
16 ----> is
132 ----> only
68 ----> one
367 ----> place
5 ----> of
1269 ----> birth
5 ----> of
2328 ----> lord
712 ----> ram
6217 ----> ayodhya
1

array([   1,  810, 5393,   10,    4,  117,   33, 2734,    2,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [ ]:
BUFFER_SIZE = len(document_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(document_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(document_tokenizer.word_index)+1
vocab_tar_size = len(summary_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((document_tensor_train, summary_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
vocab_inp_size

68462

In [ ]:
vocab_tar_size

28309

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 87]), TensorShape([64, 21]))

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 87, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 87, 1)


In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 28309)


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([summary_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 2

for epoch in tqdm(range(EPOCHS)):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in tqdm(enumerate(dataset.take(steps_per_epoch))):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))



  0%|          | 0/2 [00:00<?, ?it/s]


0it [00:00, ?it/s]


1it [01:01, 61.33s/it]

Epoch 1 Batch 0 Loss 4.8891





2it [01:30, 51.76s/it]


3it [02:00, 45.22s/it]


4it [02:30, 40.58s/it]


5it [03:00, 37.33s/it]


6it [03:30, 35.11s/it]


7it [03:59, 33.48s/it]


8it [04:29, 32.39s/it]


9it [04:59, 31.68s/it]


10it [05:29, 31.07s/it]


11it [05:59, 30.78s/it]


12it [06:30, 30.93s/it]


13it [07:02, 31.17s/it]


14it [07:33, 31.13s/it]


15it [08:04, 31.08s/it]


16it [08:35, 31.04s/it]


17it [09:06, 30.93s/it]


18it [09:36, 30.75s/it]


19it [10:09, 31.58s/it]


20it [10:41, 31.47s/it]


21it [11:11, 31.17s/it]


22it [11:43, 31.26s/it]


23it [12:14, 31.17s/it]


24it [12:45, 31.22s/it]


25it [13:15, 30.85s/it]


26it [13:45, 30.71s/it]


27it [14:15, 30.54s/it]


28it [14:46, 30.42s/it]


29it [15:15, 30.20s/it]


30it [15:45, 30.12s/it]


31it [16:15, 29.94s/it]


32it [16:44, 29.88s/it]


33it [17:14, 29.84s/it]


34it [17:45, 29.99s/it]


35it [18:12, 29.28s/it]


36it [18:42, 29.53s/it]


37it [19:12, 29.62s/it]


38it [19:42, 29.71s/it]


39it [20:12, 29.81s/it]


40it [20:44, 30.3

Epoch 1 Batch 100 Loss 3.5252





102it [48:04, 26.10s/it]


103it [48:31, 26.24s/it]


104it [48:58, 26.56s/it]


105it [49:24, 26.47s/it]


106it [49:50, 26.27s/it]


107it [50:16, 26.21s/it]


108it [50:43, 26.54s/it]


109it [51:10, 26.72s/it]


110it [51:36, 26.44s/it]


111it [52:02, 26.26s/it]


112it [52:28, 26.05s/it]


113it [52:53, 25.94s/it]


114it [53:19, 25.94s/it]


115it [53:45, 25.94s/it]


116it [54:12, 26.12s/it]


117it [54:38, 26.07s/it]


118it [55:03, 25.93s/it]


119it [55:30, 26.03s/it]


120it [55:56, 26.16s/it]


121it [56:22, 26.09s/it]


122it [56:48, 26.01s/it]


123it [57:13, 25.85s/it]


124it [57:39, 25.71s/it]


125it [58:04, 25.74s/it]


126it [58:30, 25.60s/it]


127it [58:56, 25.68s/it]


128it [59:21, 25.62s/it]


129it [59:47, 25.60s/it]


130it [1:00:14, 26.01s/it]


131it [1:00:44, 27.35s/it]


132it [1:01:16, 28.65s/it]


133it [1:01:44, 28.45s/it]


134it [1:02:14, 28.86s/it]


135it [1:02:43, 29.11s/it]


136it [1:03:13, 29.26s/it]


137it [1:03:40, 28.69s/it]


138it [1:

...




1it [00:26, 26.50s/it]

Epoch 2 Batch 0 Loss 3.1952





2it [00:52, 26.40s/it]


3it [01:18, 26.35s/it]


4it [01:44, 26.26s/it]


5it [02:11, 26.36s/it]


6it [02:37, 26.31s/it]


7it [03:04, 26.35s/it]


8it [03:30, 26.35s/it]


9it [03:57, 26.41s/it]


10it [04:23, 26.44s/it]


11it [04:49, 26.42s/it]


12it [05:16, 26.40s/it]


13it [05:42, 26.36s/it]


14it [06:08, 26.35s/it]


15it [06:35, 26.30s/it]


16it [07:03, 26.99s/it]


17it [07:29, 26.67s/it]


18it [07:56, 26.62s/it]


19it [08:22, 26.57s/it]


20it [08:48, 26.42s/it]


21it [09:14, 26.32s/it]


22it [09:41, 26.45s/it]


23it [10:07, 26.44s/it]


24it [10:34, 26.60s/it]


25it [11:01, 26.48s/it]


26it [11:27, 26.42s/it]


27it [11:53, 26.43s/it]


28it [12:20, 26.38s/it]


29it [12:46, 26.39s/it]


30it [13:12, 26.30s/it]


31it [13:38, 26.19s/it]


32it [14:04, 26.16s/it]


33it [14:30, 26.15s/it]


34it [14:56, 26.15s/it]


35it [15:22, 26.10s/it]


36it [15:49, 26.20s/it]


37it [16:15, 26.27s/it]


38it [16:42, 26.31s/it]


39it [17:10, 26.79s/it]


40it [17:36, 26.5

Epoch 2 Batch 100 Loss 3.2250





102it [44:27, 25.74s/it]


103it [44:53, 25.63s/it]


104it [45:18, 25.57s/it]


105it [45:44, 25.75s/it]


106it [46:11, 25.96s/it]


107it [46:37, 26.13s/it]


108it [47:03, 26.03s/it]


109it [47:31, 26.66s/it]